In [247]:
import numpy as np

In [248]:
n_vertices = 3
h = 20                                                      # mm
l_edge = 10                                                 # mm
alpha = 10                                                  # degrees

In [249]:
xmax = 220                                                  # mm
ymax = 220                                                  # mm
zmax = 300                                                  # mm

In [250]:
t_bed = 60                                                  # degrees C
t_ext = 210                                                 # degrees C

h_layer = 0.2                                               # mm
feed_sec = 40                                               # mm/s
t_feed_sec = 200                                            # mm/s

feed = feed_sec * 60                                        # mm/min
t_feed = t_feed_sec * 60                                    # mm/min


max_a = [500.00, 500.00, 100.00, 5000.00]
max_feed = [500.00, 500.00, 10.00, 50.00]
accs = [500.00, 1000.00, 500.00]
jerk = [8.00, 8.00, 0.40, 5.00]

v_fan = 100         # %

In [251]:
text = open('Ex_Files/Ex1.gcode', 'w')
text.write(";This Gcode was generated with GPyCode\n")
text.write(";The shape has {} vertices\n".format(int(n_vertices)))
text.write(";The print has a layer height of {} mm and the feed speed is {} mm/s\n".format(h_layer, feed_sec))


70

In [252]:
text.write('M140 S{}\n'.format(t_bed))                      # Set bed temperature without waiting
text.write('M104 S{}\n'.format(t_ext))                      # Set extruder temperature without waiting
text.write('M105\n')
text.write('M190 S{}\n'.format(t_bed))                      # Set bed temperature and wait
text.write('M109 S{}\n'.format(t_ext))                      # Set extruder temperature and wait

10

In [253]:
text.write('M82\n')
text.write('M201 X{} Y{} Z{} E{}\n'.format(*max_a))         # Set max acceleration for all axis
text.write('M203 X{} Y{} Z{} E{}\n'.format(*max_feed))      # Set max feed for all axis
text.write('M204 P{} R{} T{}\n'.format(*accs))              # Set print, retract and travel accelerations
text.write('M205 X{} Y{} Z{} E{}\n'.format(*jerk))          # Set Jerk

text.write('M220 S100\n')                                   # Feed multiplier
text.write('M221 S100\n')                                   # Flow rate multiplier

text.write('G28\n')                                         # Auto home
text.write('G92 E0\n')                                      # Reset extruder to 0
text.write('M107\n')                                        # Disable fan for first layer


5

In [254]:
xc = xmax / 2
yc = ymax / 2
text.write('G0 F{} X{} Y{} Z2.00\n'.format(t_feed, xc, yc))

30

In [255]:
# h_layer = 0.3                         # This is a test line, to be deleted
n_layers = int(h / h_layer)
z_layers = [round(h_layer + (i * h_layer), 2) for i in range(n_layers)]

if (h / h_layer) != int(h / h_layer):
    n_layers += 1
    z_layers.append(round(h, 2))

print(z_layers)
print(len(z_layers))

alpha_layers = [alpha * (i / (n_layers - 1)) for i in range(n_layers)]
print(alpha_layers)
print(len(alpha_layers))



[0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6, 2.8, 3.0, 3.2, 3.4, 3.6, 3.8, 4.0, 4.2, 4.4, 4.6, 4.8, 5.0, 5.2, 5.4, 5.6, 5.8, 6.0, 6.2, 6.4, 6.6, 6.8, 7.0, 7.2, 7.4, 7.6, 7.8, 8.0, 8.2, 8.4, 8.6, 8.8, 9.0, 9.2, 9.4, 9.6, 9.8, 10.0, 10.2, 10.4, 10.6, 10.8, 11.0, 11.2, 11.4, 11.6, 11.8, 12.0, 12.2, 12.4, 12.6, 12.8, 13.0, 13.2, 13.4, 13.6, 13.8, 14.0, 14.2, 14.4, 14.6, 14.8, 15.0, 15.2, 15.4, 15.6, 15.8, 16.0, 16.2, 16.4, 16.6, 16.8, 17.0, 17.2, 17.4, 17.6, 17.8, 18.0, 18.2, 18.4, 18.6, 18.8, 19.0, 19.2, 19.4, 19.6, 19.8, 20.0]
100
[0.0, 0.10101010101010102, 0.20202020202020204, 0.30303030303030304, 0.4040404040404041, 0.5050505050505051, 0.6060606060606061, 0.7070707070707071, 0.8080808080808082, 0.9090909090909092, 1.0101010101010102, 1.1111111111111112, 1.2121212121212122, 1.3131313131313131, 1.4141414141414141, 1.5151515151515151, 1.6161616161616164, 1.7171717171717171, 1.8181818181818183, 1.9191919191919191, 2.0202020202020203, 2.121212121212121, 2.22222222222222

In [260]:
n_vertices = 10
theta_0 = [i * (360 / n_vertices) for i in range(n_vertices)]


In [257]:
text.close()